<a href="https://colab.research.google.com/github/khuramgill/Face-Embaddings/blob/main/Testing_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_2agnRW_PJGcuYNduPh4LXPttu1bnT31UninNBGbyZqKDUiCDAxdY7asgDJkLHmR3KCDJzP")
index = pc.Index("t")

In [ ]:
import random

# Generate 200 movie vectors with 512 dimensions
movie_vectors = []
genres = ["drama", "action", "comedy", "thriller", "romance", "sci-fi", "fantasy", "horror", "documentary", "animation"]

for i in range(200):
    movie_vectors.append({
        "id": f"vec{i+1}",
        "values": [random.uniform(0, 1) for _ in range(512)],
        "metadata": {"genre": random.choice(genres)}
    })

# Upsert into Pinecone
index.upsert(
    vectors=movie_vectors,
    namespace="ns1"
)


{'upserted_count': 200}

In [ ]:
vector_ids = ["vec1", "vec2", "vec3", "vec4", "vec5"]

response = index.fetch(ids=vector_ids)

# Check if there are vectors in the response
if "vectors" in response and response["vectors"]:
    print("Fetched vectors successfully!\n")

    for vec_id, vector_data in response["vectors"].items():
        print(f"ID: {vec_id}")
        print(f"Metadata: {vector_data.get('metadata', 'No metadata available')}")

        print(f"Values (first 10 dimensions): {vector_data.get('values', [])[:10]}")
        print("\n")
else:
    print("No vectors found in the response.")


No vectors found in the response.


In [ ]:
query_vector = [random.uniform(0, 1) for _ in range(512)]

response = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)

# Display the query results
for match in response["matches"]:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadata: {match['metadata']}")
    print("\n")


In [ ]:
all_ids = [f"vec{i+1}" for i in range(200)]
batch_size = 50

for i in range(0, len(all_ids), batch_size):
    batch_ids = all_ids[i:i+batch_size]
    response = index.fetch(ids=batch_ids)

    for vec_id, vector_data in response["vectors"].items():
        print(f"ID: {vec_id}")
        print(f"Metadata: {vector_data['metadata']}")
        print(f"Values (first 10 dimensions): {vector_data['values'][:10]}")
        print("\n")


# **Now testing Image Vectors**

In [ ]:
!pip install open_clip_torch pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 2.0.1
    Uninstalling pinecone-plugin-inference-2.0.1:
      Successfully uninstalled pinecone-plugin-inference-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.0 requires pinecone-plugin-inference<3.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [ ]:
!pip install --upgrade pinecone-client


In [ ]:
import open_clip
from PIL import Image
import torch
import pinecone


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cpu


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_2agnRW_PJGcuYNduPh4LXPttu1bnT31UninNBGbyZqKDUiCDAxdY7asgDJkLHmR3KCDJzP")
index = pc.Index("t")

# Connec


load Model and its fun

In [ ]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
    'hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K'
)
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K')

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

In [48]:
image_path = '/content/28.jpg'
image = Image.open(image_path).convert('RGB')
preprocessed_image = preprocess_val(image).unsqueeze(0).to(device)


Generate Embaddings


In [40]:
with torch.no_grad():
    image_embedding = model.encode_image(preprocessed_image).cpu().numpy().flatten()


In [41]:
index.upsert(
    vectors=[
        {
            "id": "image1",
            "values": image_embedding.tolist(),
            "metadata": {"type": "image", "description": "Sample image"}
        }
    ],
    namespace="image_namespace"
)


{'upserted_count': 1}

Upload Multiple Images

In [49]:
from glob import glob

image_paths = glob('/content/sample_data/Temp/*.jpg')
vectors = []

for i, img_path in enumerate(image_paths):
    # Load and preprocess each image
    image = Image.open(img_path).convert('RGB')
    preprocessed_image = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_embedding = model.encode_image(preprocessed_image).cpu().numpy().flatten()

    vectors.append({
        "id": f"image_{i}",
        "values": image_embedding.tolist(),
        "metadata": {"type": "image", "description": f"Image {i}"}
    })

# Batch upsert to Pinecone
index.upsert(vectors=vectors, namespace="image_namespace")


{'upserted_count': 35}

Query the Index

In [52]:
query_image_path = '/content/sample_data/Temp/2.jpg'
query_image = Image.open(query_image_path).convert('RGB')
query_image = preprocess_val(query_image).unsqueeze(0).to(device)

with torch.no_grad():
    query_embedding = model.encode_image(query_image).cpu().numpy().flatten()

# Query Pinecone for similar images
response = index.query(
    vector=query_embedding.tolist(),
    top_k=5,
    include_metadata=True,
    namespace="image_namespace"
)

# results
for match in response["matches"]:
    print(f"ID: {match['id']}, Score: {match['score']}, Metadata: {match['metadata']}")


ID: image_24, Score: 1.00039065, Metadata: {'description': 'Image 24', 'type': 'image'}
ID: image_10, Score: 0.862112641, Metadata: {'description': 'Image 10', 'type': 'image'}
ID: image_15, Score: 0.8399297, Metadata: {'description': 'Image 15', 'type': 'image'}
ID: image_23, Score: 0.82713306, Metadata: {'description': 'Image 23', 'type': 'image'}
ID: image_0, Score: 0.826842427, Metadata: {'description': 'Image 0', 'type': 'image'}
